In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import gc

from itertools import product

In [2]:
sales = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
items_df = pd.read_csv('input/items.csv')
shops_df = pd.read_csv('input/shops_yakutsk.csv')
icats_df = pd.read_csv('input/item_categories.csv')

## Util

In [3]:
# downcast types to save memory
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols]   = df[int_cols].astype(np.int8)
    
    return df

## Creating of a new train

In [4]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
train = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    train.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

train = pd.DataFrame(np.vstack(train), columns = index_cols,dtype=np.int32)

In [5]:
#присоединяем цену
item_cnt_month = sales.groupby(['date_block_num', 'shop_id', 'item_id'], as_index = False)['item_cnt_day'].sum()
train = pd.merge(train, item_cnt_month, on = ['date_block_num', 'shop_id', 'item_id'], how = 'left')
train.rename(columns = {'item_cnt_day' : 'item_cnt_month'}, inplace = True)

train['item_cnt_month'] = train['item_cnt_month'].fillna(0)

In [6]:
#add 34th month
test["date_block_num"] = train['date_block_num'].max() + 1

## Add info from addition files

In [7]:
#preparing of additional tables
def take_last(x):
    s = x.split('-')
    if len(s) > 1:
        return s[-1]
    return ''

#items
subcategory = icats_df['item_category_name'].apply(take_last)
unique_splt = subcategory.unique()
submapped = dict(zip(unique_splt , range(len(unique_splt)))) 
icats_df['subcategory'] = subcategory.map(submapped)

supercategory = icats_df['item_category_name'].apply(lambda x: x.split('-')[0])
unique_splt = supercategory.unique()
supermapped = dict(zip(unique_splt , range(len(unique_splt)))) 
icats_df['supercategory'] = supercategory.map(supermapped)

#shop
city = shops_df['shop_name'].apply(lambda x: x.split(' ')[0])
unique_splt = city.unique()
city_map = dict(zip(unique_splt , range(len(unique_splt)))) 
shops_df['city'] = city.map(city_map)

shop_type = shops_df['shop_name'].apply(lambda x: x.split(' ')[1])
unique_splt = shop_type.unique()
shop_type_map = dict(zip(unique_splt , range(len(unique_splt)))) 
shops_df['shop_type'] = shop_type.map(shop_type_map)

In [8]:
#merge
items_merged_cats = pd.merge(items_df, icats_df, on = 'item_category_id', how = 'left')

train = pd.merge(train, items_merged_cats, on = 'item_id', how = 'left')
test = pd.merge(test, items_merged_cats, on = 'item_id', how = 'left')
train = pd.merge(train, shops_df, on = 'shop_id', how = 'left')
test = pd.merge(test, shops_df, on = 'shop_id', how = 'left')

In [9]:
#drop extra
#item_name пока удалять не стоит, тк в них можно поискать паттерны. по типу: 'аудиокниги', 'образовательная литература'
train.drop(labels = ['item_category_name', 'shop_name', 'item_name'], axis = 1, inplace = True)
test.drop(labels = ['item_category_name', 'shop_name', 'ID', 'item_name'], axis = 1, inplace = True)

## Add features

### summary

In [10]:
summary_shop_sales = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'summary_shop_sales':'sum'}})
summary_shop_sales.columns = [col[0] if col[-1]=='' else col[-1] for col in summary_shop_sales.columns.values]

summary_item_sales = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'summary_item_sales':'sum'}})
summary_item_sales.columns = [col[0] if col[-1] == '' else col[-1] for col in summary_item_sales.columns.values]

summary_sales = sales.groupby(['date_block_num'],as_index=False).agg({'item_cnt_day':{'summary_sales':'sum'}})
summary_sales.columns = [col[0] if col[-1] == '' else col[-1] for col in summary_sales.columns.values]

/home/whale/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


### mean

In [11]:
mean_sales = sales.groupby(['date_block_num'],as_index=False).agg({'item_cnt_day':{'mean_sales':'mean'}})
mean_sales.columns = [col[0] if col[-1] == '' else col[-1] for col in mean_sales.columns.values]

mean_shop_sales = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'mean_shop_sales':'mean'}})
mean_shop_sales.columns = [col[0] if col[-1]=='' else col[-1] for col in mean_shop_sales.columns.values]

mean_item_sales = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'mean_item_sales':'mean'}})
mean_item_sales.columns = [col[0] if col[-1] == '' else col[-1] for col in mean_item_sales.columns.values]

/home/whale/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [12]:
#merge with the train
train = pd.merge(train, summary_shop_sales, how='left', on=['shop_id', 'date_block_num']).fillna(0)
train = pd.merge(train, summary_item_sales, how='left', on=['item_id', 'date_block_num']).fillna(0)
train = pd.merge(train, summary_sales, how='left', on=['date_block_num']).fillna(0)
train = pd.merge(train, mean_shop_sales, how='left', on=['shop_id', 'date_block_num']).fillna(0)
train = pd.merge(train, mean_item_sales, how='left', on=['item_id', 'date_block_num']).fillna(0)
train = pd.merge(train, mean_sales, how='left', on=['date_block_num']).fillna(0)


test = pd.merge(test, summary_shop_sales, how='left', on=['shop_id', 'date_block_num']).fillna(0)
test = pd.merge(test, summary_item_sales, how='left', on=['item_id', 'date_block_num']).fillna(0)
test = pd.merge(test, summary_sales, how='left', on=['date_block_num']).fillna(0)
test = pd.merge(test, mean_shop_sales, how='left', on=['shop_id', 'date_block_num']).fillna(0)
test = pd.merge(test, mean_item_sales, how='left', on=['item_id', 'date_block_num']).fillna(0)
test = pd.merge(test, mean_sales, how='left', on=['date_block_num']).fillna(0)

### shift

In [13]:
all_data = pd.concat([train, test])

/home/whale/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


### free up memory

In [14]:
del train, test, sales, items_df, shops_df, icats_df, item_cnt_month, items_merged_cats, summary_shop_sales,
del summary_item_sales, summary_sales, mean_shop_sales, mean_item_sales
gc.collect()

267

In [15]:
# List of columns that we will be used to create shift
cols_to_rename = ['item_cnt_month', 'summary_item_sales', 'summary_shop_sales']

for month_shift in [1, 2, 3, 4]:
    data_shift = all_data[index_cols + cols_to_rename].copy()
    #взять 10 месяц из олдаты, сдвинуть его вперед(например до 11-го) и смерджить 11 получившийся месяц к 10-у
    data_shift['date_block_num'] = data_shift['date_block_num'] + month_shift
    #change name
    foo = lambda x: '{}_shift_{}'.format(x, month_shift) if x in cols_to_rename else x
    data_shift = data_shift.rename(columns = foo)
    all_data = pd.merge(all_data, data_shift, on = index_cols, how = 'left')

In [16]:
train = all_data[all_data['date_block_num'] != (all_data['date_block_num'].max())]
test = all_data[all_data['date_block_num'] == (all_data['date_block_num'].max())]

In [17]:
# I shouldn't use old data from year 2013
train = train[(train["date_block_num"] > 20)]

In [18]:
train.drop(labels = ['summary_item_sales', 'summary_shop_sales'], axis = 1, inplace = True)
test.drop(labels = ['summary_item_sales', 'summary_shop_sales', 'item_cnt_month'], axis = 1, inplace = True)

/home/whale/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Write down

In [19]:
train.to_csv('generated/train.csv', index=False)
test.to_csv('generated/test.csv', index=False)